<a href="https://colab.research.google.com/github/AtlastDeepLearning/Intestinal-Parasite-Classification-LW-CNN/blob/main/Intestinal_Parasite_LW_CNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Environment Setup in Google Colab
1.1 Enable GPU Runtime
In Colab: Runtime → Change runtime type → GPU to accelerate training **

In [ ]:
!pip install tensorflow==2.12.0              # Colab’s TF version
!pip install tflite-runtime                # For local TFLite tests
!pip install efficientnet                  # Provides EfficientNetB0–B7
!pip install imgaug albumentations         # Data augmentation (optional)

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 7.6 MB/s eta 0:00:00
  Atte

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.12.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.12.0 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, 

# 2. Data Preparation
**2.1 Folder Structure [warning do not run cell below]**

In [ ]:
/content/drive/MyDrive/parasites/
├── train/
│   ├── ascaris_lumbricoides/
│   ├── trichuris_trichiura/
│   ├── hookworms/
│   ├── enterobius_vermicularis/
│   └── giardia_lamblia/
└── val/
    ├── ascaris_lumbricoides/
    ├── trichuris_trichiura/
    ├── hookworms/
    ├── enterobius_vermicularis/
    └── giardia_lamblia/

**2.2 (Optional) Augmentation**

In [ ]:
from albumentations import RandomRotate90, Flip, Normalize
# set up augmentation pipeline to increase dataset variety

# 3. Building the Transfer‑Learning Model
**3.1 Import and Initialize EfficientNetB0**

In [ ]:
from efficientnet.tfkeras import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))

**3.2 Add Custom Classification Head**

In [ ]:
import tensorflow as tf

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

**3.3 Compile the Model**

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

**Use a low LR for fine‑tuning**

# 4. Training & Evaluation
**4.1 Data Generators**

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(...)
val_gen   = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(...)

**4.2 Fit the Model**

In [ ]:
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(), tf.keras.callbacks.EarlyStopping(patience=5)]
)

**Monitor for overfitting and adjust epochs accordingly**

**4.3 Evaluate Performance**

In [ ]:
model.evaluate(val_gen)

# 5. Model Conversion to TensorFlow Lite
**5.1 Save the Keras Model**

In [ ]:
model.save('parasite_classifier.h5')

**5.2 Convert to TFLite with Quantization**

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('parasite_classifier.h5')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('parasite_classifier.tflite','wb').write(tflite_model)

# Code for the raspbery pi app

In [ ]:
import cv2, numpy as np
from tflite_runtime.interpreter import Interpreter

# Load TFLite model
interpreter = Interpreter(model_path='parasite_classifier.tflite')
interpreter.allocate_tensors()

# Get I/O details
input_details  = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Capture from camera
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224,224))
    inp = np.expand_dims(img.astype(np.float32)/255.0, axis=0)
    interpreter.set_tensor(input_details[0]['index'], inp)
    interpreter.invoke()
    preds = interpreter.get_tensor(output_details[0]['index'])[0]
    class_id = np.argmax(preds)
    # map class_id to parasite name and display
    cv2.putText(frame, names[class_id], (10,30), ...)
    cv2.imshow('Parasite Classifier', frame)
    if cv2.waitKey(1)==27: break
cap.release(); cv2.destroyAllWindows()